In [1]:
import os
import argparse
import time
from datetime import datetime, date
import random

import numpy as np
from scipy.sparse import load_npz
from scipy.stats import pearsonr
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import pandas as pd

import torch
import torch_geometric
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from model_classes_ import GCN_classification, GCN_regression, MLP_Classification, MLP_Regression, CNN
from custom_funcs import experiment_n

In [2]:
torch.cuda.empty_cache()

In [3]:
# Check for GPU
print(torch.cuda.is_available())  # True/False
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.current_device())  # Current GPU ID (e.g., `0`)
print(torch.cuda.get_device_name(0))  # GPU name

True
1
0
NVIDIA GeForce RTX 2060


In [4]:
# Hyperparameters
chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)
regression_flag = 0
max_epoch = 50
learning_rate = 0.001
num_lin_layers = 2
lin_hidden_size = 100
num_graph_conv_layers = 2
graph_conv_embed_size = 256
num_runs = 10
graph_conv_layer_sizes = [num_feat] + \
        [int(max(graph_conv_embed_size, lin_hidden_size)) \
              for i in np.arange(1, num_graph_conv_layers, 1)] + [lin_hidden_size]

lin_hidden_sizes_r = [graph_conv_layer_sizes[-1]] + \
        [int(max(lin_hidden_size, 1)) \
              for i in np.arange(1, num_lin_layers, 1)] + [1]
lin_hidden_sizes_c = [graph_conv_layer_sizes[-1]] + \
        [int(max(lin_hidden_size, 2)) \
              for i in np.arange(1, num_lin_layers, 1)] + [2]

In [5]:
# Stores AUROC and PCC across all models for each cell line
E116 = {'name': 'E116', 'AUROC': None, 'PCC': None}
E122 = {'name': 'E122', 'AUROC': None, 'PCC': None}
E123 = {'name': 'E123', 'AUROC': None, 'PCC': None}

In [6]:
for cell_line in [E116, E122, E123]:
        print(f"Cell line: {cell_line['name']}")
        experiment_n(
            cell_line, 
            num_runs, 
            GCN_classification(num_feat, num_graph_conv_layers, graph_conv_layer_sizes, num_lin_layers, lin_hidden_sizes_c, 2),
            GCN_regression(num_feat, num_graph_conv_layers, graph_conv_layer_sizes, num_lin_layers, lin_hidden_sizes_r, 1),
            'GCN', 
            max_epoch=max_epoch
        )

Cell line: E116
Classification Task...
Iteration 1:


Epoch 0 out of 50


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.73 GiB (GPU 0; 6.00 GiB total capacity; 1.19 GiB already allocated; 0 bytes free; 4.47 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF